## SDX-Fabric API
AtlanticWave SDX Cross-Domain Orchestration and Provisioning for Facilities and R&E Networks user interfaces with the SDXLib in Fabric Jupyter Notebook

This notebook experiment emphasizes the building of Layer 2 VPNs and the use of Fabric Facility Ports for network orchestration and provisioning across domains. It shows the integration of SDX Lib with the SDX Controller by using token-based authentication, closing the security gap and promoting safe communication between SDX Lib and the SDX Controller.

## Configure the Environment
If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.    
If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. 

## Step 1:  Authentication & Token Handling

Obtain the Token
When a user logs into FABRIC Jupyter Hub, the system automatically obtains and stores a token.

Environment variable: $FABRIC_TOKEN_LOCATION
Default path: /home/fabric/.tokens.json

## Import the FABlib Library and get the id token

## Session/Auth

SDXClient(timeout=6.0, token=None, session=None)
Creates a client bound to BASE_URL. If no token is given, tries to load it with fablib. Keeps a requests.Session for reuse.

set_token(token)
Replace or inject the Bearer token at runtime.

## Discovery (read-only)

get_topology()
GET /topology. Returns raw topology JSON (as served by the API).

get_available_ports(search=None, filter=None, limit=None, fields=None, format="html")
GET /available_ports. Lists customer-facing ports with VLAN availability.

filter: plain, case-insensitive substring across Port ID / Device / Port (space terms ANDed).

search: key:value tokens + plain terms.

If both are present, filter wins.

format="html" → HTML table (string). format="json" → structured JSON.

get_all_vlans_available()
GET /available_vlans?format=json. Bulk compact ranges per port (e.g., ["100-120","300"]).

get_port_vlans_available(port_id)
GET /available_vlans?port_id=...&format=json. VLAN ranges for a specific port.

## Selection (stateful)

begin_l2vpn_selection() / clear_selection()
Reset the internal selection: first=None, second=None.

get_selected_endpoints()
Peek at what’s currently selected ({"first": ..., "second": ...}).

set_endpoint(endpoint_position, *, filter=None, search=None, port_id=None, vlan=None, prefer_untagged=False)
Pick and store the "first" or "second" endpoint. Two paths:

Direct: supply port_id (URN) → fetch /device_info → pick a VLAN (or use vlan if given).

Text: supply filter or search → list /available_ports?format=json → narrow candidates → fetch /device_info for the chosen row → pick a VLAN.
Ambiguity returns status_code: 0 and a short candidate list.
prefer_untagged=True asks the picker to choose untagged if offered.

## Payload & L2VPN CRUD (via routes)

preview_l2vpn_payload(name, notifications)
Returns the JSON body that POST /l2vpn will receive. Requires both endpoints selected.

create_l2vpn_from_selection(name, notifications)
Builds payload (via preview_l2vpn_payload) and posts to POST /l2vpn.

get_l2vpns(**query)
GET /l2vpns with optional filters (e.g., archived, search).

get_l2vpn(service_id)
GET /l2vpn/<service_id>.

update_l2vpn(service_id, **fields)
PATCH /l2vpn/<service_id>.

delete_l2vpn(service_id)
DELETE /l2vpn/<service_id>.

## Tips

If you pass both filter and search, filter is used.

Ambiguous text matches: you’ll get {status_code: 0, data: {"candidates": [...]}}. Refine text or use exact port_id.

prefer_untagged=True only matters when untagged is advertised for that port; otherwise the first available token/range is used.

For machine workflows, favor format="json" and parse the ports array.

# Check if sdxclient is installed, uninstall it if needed

In [1]:
!pip show sdxclient
#!pip uninstall -y sdxlib  # Uninstall if installed

In [2]:
!pip install sdxclient

## Instantiate the SDXclient

In [3]:
from IPython.display import HTML
from sdxclient.client import SDXClient
client = SDXClient()

User: lmarinve@fiu.edu bastion key is valid!
Configuration is valid


## List available ports with server-side filtering (search, limit).

In [4]:
HTML(client.get_available_ports(format="html", limit=20)["data"])

Domain,Device,Port,Status,Port ID,Entities,VLANs Available,VLANs in Use
sax.net,FOR-ACB-SW01,et-0/1/0,up,urn:sdx:port:sax.net:FOR-ACB-SW01:et-0/1/0,,1-4095,None
sax.net,FOR-ACB-SW01,et-0/1/1,up,urn:sdx:port:sax.net:FOR-ACB-SW01:et-0/1/1,,1-4095,None
sax.net,FOR-ACB-SW01,et-1/1/0,up,urn:sdx:port:sax.net:FOR-ACB-SW01:et-1/1/0,,1-4095,None
sax.net,FOR-ACB-SW01,et-1/1/8,up,urn:sdx:port:sax.net:FOR-ACB-SW01:et-1/1/8,,1-4095,None
sax.net,FOR-ACB-SW01,ae1,up,urn:sdx:port:sax.net:FOR-ACB-SW01:ae1,,1-4095,None
sax.net,FOR-ACB-SW01,ae3,up,urn:sdx:port:sax.net:FOR-ACB-SW01:ae3,,1-4095,None
sax.net,FOR-LAN-SW02,et-0/0/1,up,urn:sdx:port:sax.net:FOR-LAN-SW02:et-0/0/1,,1-4095,None
sax.net,FOR-LAN-SW02,et-0/0/2,up,urn:sdx:port:sax.net:FOR-LAN-SW02:et-0/0/2,,1-4095,None
sax.net,FOR-LAN-SW02,ae1,up,urn:sdx:port:sax.net:FOR-LAN-SW02:ae1,,1-4095,None
amlight.net,MIA-MI1-SW14,9,up,urn:sdx:port:amlight.net:MIA-MI1-SW14:9,,1-4095,None


### Filter Entities

In [5]:
HTML(client.get_available_ports(format="html", fields="Domain,Device,Port,Status,Port ID,Entities", limit=20)["data"])

Domain,Device,Port,Status,Port ID,Entities
amlight.net,MIA-MI1-SW17,7,up,urn:sdx:port:amlight.net:MIA-MI1-SW17:7,FABRIC FIU - Miami
amlight.net,MIA-MI1-SW17,12,up,urn:sdx:port:amlight.net:MIA-MI1-SW17:12,NRP-k8s-gen4-02
amlight.net,MIA-MI1-SW17,21,up,urn:sdx:port:amlight.net:MIA-MI1-SW17:21,NRP-k8s-ceph-01-eno1
amlight.net,MIA-MI1-SW17,22,up,urn:sdx:port:amlight.net:MIA-MI1-SW17:22,NRP-k8s-ceph-02-eno1
amlight.net,MIA-MI1-SW17,23,up,urn:sdx:port:amlight.net:MIA-MI1-SW17:23,NRP-k8s-gen4-01-eno2
amlight.net,MIA-MI1-SW17,27,up,urn:sdx:port:amlight.net:MIA-MI1-SW17:27,Telemetry Feed
amlight.net,MIA-MI1-SW18,8,up,urn:sdx:port:amlight.net:MIA-MI1-SW18:8,AmLight DTN - Miami - OX Server


### Get Vlans in use

In [6]:
HTML(client.get_available_ports(format="html", fields="Domain,Device,Port,Status,Port ID,Entities,VLANs in Use", limit=20)["data"])

Domain,Device,Port,Status,Port ID,Entities,VLANs in Use
amlight.net,MIA-MI1-SW17,7,up,urn:sdx:port:amlight.net:MIA-MI1-SW17:7,FABRIC FIU - Miami,4015
amlight.net,MIA-MI1-SW18,8,up,urn:sdx:port:amlight.net:MIA-MI1-SW18:8,AmLight DTN - Miami - OX Server,2993


### Get Vlans available

In [7]:
HTML(client.get_available_ports(format="html", fields="Domain,Device,Port,Status,Port ID,Entities,VLANs Available", limit=20)["data"])

Domain,Device,Port,Status,Port ID,Entities,VLANs Available
amlight.net,MIA-MI1-SW17,7,up,urn:sdx:port:amlight.net:MIA-MI1-SW17:7,FABRIC FIU - Miami,"2990-2999, 4016-4094"
amlight.net,MIA-MI1-SW17,12,up,urn:sdx:port:amlight.net:MIA-MI1-SW17:12,NRP-k8s-gen4-02,1-4095
amlight.net,MIA-MI1-SW17,21,up,urn:sdx:port:amlight.net:MIA-MI1-SW17:21,NRP-k8s-ceph-01-eno1,1-4095
amlight.net,MIA-MI1-SW17,22,up,urn:sdx:port:amlight.net:MIA-MI1-SW17:22,NRP-k8s-ceph-02-eno1,1-4095
amlight.net,MIA-MI1-SW17,23,up,urn:sdx:port:amlight.net:MIA-MI1-SW17:23,NRP-k8s-gen4-01-eno2,"2990-2999, 4015-4019"
amlight.net,MIA-MI1-SW17,27,up,urn:sdx:port:amlight.net:MIA-MI1-SW17:27,Telemetry Feed,1-4095
amlight.net,MIA-MI1-SW18,8,up,urn:sdx:port:amlight.net:MIA-MI1-SW18:8,AmLight DTN - Miami - OX Server,"2990-2992, 2994-2999"


### Search device

In [8]:
HTML(client.get_available_ports(format="html", search="MI1", limit=20)["data"])

Domain,Device,Port,Status,Port ID,Entities,VLANs Available,VLANs in Use
amlight.net,MIA-MI1-SW14,9,up,urn:sdx:port:amlight.net:MIA-MI1-SW14:9,,1-4095,None
amlight.net,MIA-MI1-SW14,13,up,urn:sdx:port:amlight.net:MIA-MI1-SW14:13,,1-4095,None
amlight.net,MIA-MI1-SW14,19,up,urn:sdx:port:amlight.net:MIA-MI1-SW14:19,,1-4095,None
amlight.net,MIA-MI1-SW14,20,up,urn:sdx:port:amlight.net:MIA-MI1-SW14:20,,1-4095,None
amlight.net,MIA-MI1-SW14,23,up,urn:sdx:port:amlight.net:MIA-MI1-SW14:23,,1-4095,None
amlight.net,MIA-MI1-SW14,114,up,urn:sdx:port:amlight.net:MIA-MI1-SW14:114,,1-4095,None
amlight.net,MIA-MI1-SW15,9,up,urn:sdx:port:amlight.net:MIA-MI1-SW15:9,,"2990-2999, 4015-4019",None
amlight.net,MIA-MI1-SW15,10,up,urn:sdx:port:amlight.net:MIA-MI1-SW15:10,,1-4095,None
amlight.net,MIA-MI1-SW15,13,up,urn:sdx:port:amlight.net:MIA-MI1-SW15:13,,1-4095,None
amlight.net,MIA-MI1-SW15,15,up,urn:sdx:port:amlight.net:MIA-MI1-SW15:15,,2840-2841,None


### Search Fabric

In [9]:
HTML(client.get_available_ports(format="html", search="fabric", limit=20)["data"])

Domain,Device,Port,Status,Port ID,Entities,VLANs Available,VLANs in Use
amlight.net,MIA-MI1-SW17,7,up,urn:sdx:port:amlight.net:MIA-MI1-SW17:7,FABRIC FIU - Miami,"2990-2999, 4016-4094",4015


## Troubleshooting & Error examples

## Missing parameter

In [10]:
client.get_port_vlans_available()   # no port_id

{'status_code': 0,
 'data': None,
 'error': 'missing required parameter(s): port_id'}

## Correct usage

In [11]:
client.get_port_vlans_available("urn:sdx:port:amlight.net:MIA-MI1-SW17:27")

{'status_code': 200,
 'data': {'count': 1,
  'status_code': 200,
  'vlans': [{'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW17:27',
    'vlans_available': ['1-4095']}]},
 'error': None}

## Get All VLANs

In [ ]:
# client.get_all_vlans_available()

## Clears any previously stored endpoints

In [12]:
client.clear_selection()

{'status_code': 200,
 'data': {'name': None,
  'notifications': None,
  'first': None,
  'second': None,
  'service_id': None},
 'error': None}

## SET ENDPOINTS

## Set Endpoint Ambiguous search/filter

In [13]:
client.set_endpoint(endpoint_position="first", filter="SW17")

{'status_code': 0,
 'data': {'candidates': ['urn:sdx:port:amlight.net:MIA-MI1-SW17:6',
   'urn:sdx:port:amlight.net:MIA-MI1-SW17:7',
   'urn:sdx:port:amlight.net:MIA-MI1-SW17:8',
   'urn:sdx:port:amlight.net:MIA-MI1-SW17:10',
   'urn:sdx:port:amlight.net:MIA-MI1-SW17:12',
   'urn:sdx:port:amlight.net:MIA-MI1-SW17:13',
   'urn:sdx:port:amlight.net:MIA-MI1-SW17:14',
   'urn:sdx:port:amlight.net:MIA-MI1-SW17:15']},
 'error': 'ambiguous filter/search matched 22 ports; refine or use exact port_id'}

## Set Endpoint wrong port_id

In [14]:
client.set_endpoint(endpoint_position="second", port_id="urn:...:SW17:27", vlan="4094")

{'status_code': 400,
 'data': {'error': "missing 'device' query parameter", 'status_code': 400},
 'error': "missing 'device' query parameter"}

## Set Endpoint VLAN not available on port

In [15]:
client.set_endpoint(endpoint_position="second", port_id="urn:sdx:port:amlight.net:MIA-MI1-SW15:15", vlan="4094")

{'status_code': 0,
 'data': {'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW15:15',
  'available': ['2840-2841']},
 'error': "VLAN '4094' not available on port 'urn:sdx:port:amlight.net:MIA-MI1-SW15:15'"}

## Missing required name

In [16]:
client.set_endpoint(endpoint_position="first", filter="SW17:7", vlan=2990)
client.set_endpoint(endpoint_position="second", filter="SW17:7", vlan=2990)
client.get_l2vpn_payload()

{'status_code': 0,
 'data': None,
 'error': 'missing L2VPN name (set via set_l2vpn_payload)'}

# used VLAN not Available

In [17]:
client.set_endpoint(endpoint_position="first", filter="SW17:7", vlan=2992)
client.set_endpoint(endpoint_position="second", filter="SW17:7", vlan=2992)

{'status_code': 200,
 'data': {'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW17:7',
  'vlan': '2992'},
 'error': None}

## Invalid Same VLAN

In [18]:
client.set_endpoint(endpoint_position="first", filter="SW17:7", vlan=2997)
client.set_endpoint(endpoint_position="second", filter="SW17:7", vlan=2997)
client.set_l2vpn_payload(name="test-sdxlib-mia", notifications="info@example.org")
client.get_l2vpn_payload()

{'status_code': 0,
 'data': None,
 'error': 'endpoints cannot use the same numeric VLAN (2997) twice.'}

In [19]:
client.get_l2vpn_payload()

{'status_code': 0,
 'data': None,
 'error': 'endpoints cannot use the same numeric VLAN (2997) twice.'}

## Get L2VPNs

In [20]:
client.get_l2vpns()

{'status_code': 200,
 'data': {'count': 1,
  'l2vpns': [{'Name': 'DDoS w p4',
    'Notifications': 'absahin@fiu.edu',
    'Ownership': '',
    'QoS Metrics': 'None',
    'Scheduling': 'None',
    'Service ID': 'b650fefc-49a5-46e7-9206-70cab5e215e0',
    'endpoints': [{'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW17:7',
      'vlan': '4015'},
     {'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW18:8', 'vlan': '2993'}]}],
  'status_code': 200},
 'error': None}

## Clears any previously stored endpoints

In [21]:
client.clear_selection()

{'status_code': 200,
 'data': {'name': None,
  'notifications': None,
  'first': None,
  'second': None,
  'service_id': None},
 'error': None}

## SET FIRST ENDPOINT

In [22]:
client.set_endpoint(endpoint_position="first", filter="SW17:7", vlan=2995)

{'status_code': 200,
 'data': {'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW17:7',
  'vlan': '2995'},
 'error': None}

## SET SECOND ENDPOINT

In [23]:
client.set_endpoint(endpoint_position="second", filter="SW17:27", vlan=2996)

{'status_code': 200,
 'data': {'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW17:27',
  'vlan': '2996'},
 'error': None}

## Preview the L2VPN payload

In [24]:
client.get_l2vpn_payload()

{'status_code': 0,
 'data': None,
 'error': 'missing L2VPN name (set via set_l2vpn_payload)'}

## SET the L2VPN payload

In [25]:
client.set_l2vpn_payload(name="test-sdxlib-mia", notifications=[{"email":"info@example.org"}])

{'status_code': 200,
 'data': {'name': 'test-sdxlib-mia',
  'endpoints': [{'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW17:7',
    'vlan': '2995'},
   {'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW17:27', 'vlan': '2996'}],
  'notifications': [{'email': 'info@example.org'}]},
 'error': None}

## Get current stored parameters

In [26]:
client.get_selection()

{'status_code': 200,
 'data': {'name': 'test-sdxlib-mia',
  'notifications': [{'email': 'info@example.org'}],
  'first': {'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW17:7',
   'vlan': '2995'},
  'second': {'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW17:27',
   'vlan': '2996'},
  'service_id': None},
 'error': None}

## Create L2VPN

In [27]:
client.create_l2vpn_from_selection()

{'data': {'reason': 'Connection published',
  'service_id': 'c827583e-d4b0-4066-8bb9-6b708fa02a09',
  'status': 'under provisioning'},
 'error': None,
 'status_code': 201}

## Get current stored parameters

In [28]:
client.get_selection()

{'status_code': 200,
 'data': {'name': 'test-sdxlib-mia',
  'notifications': [{'email': 'info@example.org'}],
  'first': {'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW17:7',
   'vlan': '2995'},
  'second': {'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW17:27',
   'vlan': '2996'},
  'service_id': 'c827583e-d4b0-4066-8bb9-6b708fa02a09'},
 'error': None}

## Fetch the created L2VPN by service_id

In [29]:
client.get_l2vpn()

{'data': {'c827583e-d4b0-4066-8bb9-6b708fa02a09': {'archived_date': 0,
   'current_path': [{'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW17:7',
     'vlan': '2995'},
    {'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW17:27', 'vlan': '2996'}],
   'description': None,
   'endpoints': [{'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW17:7',
     'vlan': '2995'},
    {'port_id': 'urn:sdx:port:amlight.net:MIA-MI1-SW17:27', 'vlan': '2996'}],
   'name': 'test-sdxlib-mia',
   'notifications': [{'email': 'info@example.org'}],
   'oxp_response': {'amlight.net': [200,
     {'circuit_id': '35315d0f36c540', 'deployed': True}]},
   'service_id': 'c827583e-d4b0-4066-8bb9-6b708fa02a09',
   'status': 'up'}},
 'error': None,
 'status_code': 200}

## Delete the created L2VPN by service_id

In [30]:
client.delete_l2vpn()

{'data': 'OK', 'error': None, 'status_code': 200}